### 1 - Imports

In [366]:
import os
import tiktoken
import difflib
import pandas as pd
import io
import sys
from dotenv import load_dotenv
from firecrawl import FirecrawlApp
from langchain_openai import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
import requests
import urllib.parse


### 2 - OpenAI Azure models deployement

In [3]:
load_dotenv()

True

In [4]:

llm_4o = AzureChatOpenAI(
    azure_endpoint=os.getenv('Azure_OpenAI_OB_Endpoint'), 
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4o",
    openai_api_key=os.getenv('Azure_OpenAI_OB_Key'), 
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt-4o-deploy")
    #streaming=True

In [5]:
llm_4omini = AzureChatOpenAI(
    azure_endpoint=os.getenv('Azure_OpenAI_OB_Endpoint_4mini'), 
    openai_api_version="2024-10-21",   
    model_name="gpt-4o-mini",
    openai_api_key=os.getenv('Azure_OpenAI_OB_Key_4mini'), 
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt4o-mini")
    #streaming=True

In [6]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=os.getenv('Azure_OpenAI_emb_3_large_azure_endpoint'),
    openai_api_key=os.getenv('Azure_OpenAI_emb_3_large_api_key'),
    openai_api_version="2023-05-15",
    chunk_size=1000
)


/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_1826/426928708.py:1: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embeddings = AzureOpenAIEmbeddings(


In [7]:
prompt = "Fais une blague sur les actuaires"

for chunk in llm_4o.stream(prompt):
    print(chunk.content, end="", flush=True)

Pourquoi les actuaires ne racontent-ils jamais de blagues ?

Parce que la probabilité que quelqu'un les trouve drôles est trop faible !

In [8]:
for chunk in llm_4omini.stream(prompt):
    print(chunk.content, end="", flush=True)

Pourquoi les actuaires ne racontent-ils jamais de blagues ?

Parce qu'ils préfèrent les statistiques à l'humour, et que les rires ne sont pas toujours prévisibles !

In [298]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
emb1 = embeddings.embed_query(sentence1)
emb2 = embeddings.embed_query(sentence2)
print(emb1)
print(emb2)

[-0.020320029975815345, -0.014629864167398414, -0.007168493715699034, -0.0053693973582268385, 0.022537522069569143, 0.012119496268966847, -0.012935365579843398, -0.0030996066123450973, -0.0017677171894784922, 0.040416918787532105, -0.022007554819915672, -0.009253493531378172, -0.014880901236638343, -0.025591801910595832, -0.022425949624874694, -0.009971737884469659, 0.0025888164587000947, 0.01482511501429112, 0.0006964526853641184, -0.0014059800919678843, 0.02783718711552324, -0.025145513994463197, -0.0017311075974784488, 0.033220529632353016, 0.022718826360875043, 0.023932170177669118, -0.0038143640937092376, -0.01877197068758308, -0.008249346185741031, -0.035284608683329374, -0.006851211205913708, 0.031519059163988464, -0.017795717384442127, 0.034419927824882866, 0.021505482544080964, -0.044182468306873016, 0.03768340506838907, -0.01421146936243939, -0.003985208468325033, 0.02647043025595172, 0.02266304013852782, -0.007698460499691216, -0.019413508519285846, -0.03997062900875432, 0.0

### 3 -  Data pipeline 

In [9]:
def count_tokens(text: str, model: str = "gpt-4o") -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

#### 3.1 -  Data importing

#### 3.1.1 -  Firecrawl (URL -> Markdown , HTML)

In [324]:
url_aml_5="http://publications.europa.eu/resource/celex/32015L0849"
url_crr="http://publications.europa.eu/resource/celex/32013R0575"

In [10]:
app = FirecrawlApp(api_key=os.getenv('api_key_firecraw_2'))

In [300]:
scrape_result_crr = app.scrape_url(
    url_crr,
    params={
        'formats': ['markdown', 'html'],
        'location': {
            'country': 'FR'
        }
    }
)

In [ ]:
scrape_result_AML_5 = app.scrape_url(
    url_aml_5,
    params={
        'formats': ['markdown', 'html'],
        'location': {
            'country': 'FR'
        }
    }
)

In [ ]:
directive_UE_AML5 =  scrape_result_AML_5["markdown"] 
count_tokens(directive_UE_AML5)

50960

In [250]:
directive_UE_CRR =  scrape_result_crr["markdown"]
count_tokens(directive_UE_CRR)

408669

#### 3.1.2 -  URL to PDF

In [373]:
def url_to_pdf(url, save_path):
    response = requests.get(url)
    with open(save_path, "wb") as f:
        f.write(response.content)

In [ ]:
url_to_pdf("https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32013R0575", 
                "/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_input/test.pdf")

#### 3.1.3 -  PDF loader

In [ ]:

def charger_pdf(chemin_pdf):
    loader = PyPDFLoader(chemin_pdf)
    documents = loader.load()
    return documents

documents = charger_pdf(chemin_pdf)

#### 3.2 -  Data cleaning / token optimization

Le contenu de la directive commence par : "ONT ADOPTÉ LA PRÉSENTE DIRECTIVE:\n\nCHAPITRE I"

Et se termine avant "Par le Parlement européen\n\nLe président\"

Solution : On supprime donc le contenu avant cette phrase avec les deux fonctions : supr_avant_directive + supr_apres_directive

#### 3.2.1 - directive pipeline cleaning

In [253]:
def supr_avant_directive(texte):
    start_phrase = "ONT ADOPTÉ LA PRÉSENTE DIRECTIVE:\n\nCHAPITRE I"
    index_debut = texte.find(start_phrase)
    if index_debut != -1:
        return texte[index_debut:]
    else:
        return "Le texte ne contient pas la phrase spécifiée."

In [22]:
def supr_apres_directive(texte):
    end_phrase = "Par le Parlement européen\n\nLe président\n"
    index_fin = texte.find(end_phrase)
    if index_fin != -1:
        return texte[:index_fin + len(end_phrase)]
    else:
        return texte  

In [23]:
directive_UE_step_1=supr_avant_directive(directive_UE)
count_tokens(directive_UE_step_1) # gain de 13K token

37286

In [24]:
directive_UE_step_2=supr_apres_directive(directive_UE_step_1)
count_tokens(directive_UE_step_2) # gain de 8K/9k token

29819

Optionnel : Suppression des caractères markdown

In [26]:
import re

def nettoyer_markdown(texte):
    # Supprimer les tableaux Markdown
    texte = re.sub(r'\|.*?\|', '', texte)
    texte = re.sub(r'\|\s*---\s*\|', '', texte)
    
    # Supprimer les liens Markdown
    texte = re.sub(r'\[.*?\]\(.*?\)', '', texte)
    
    # Supprimer les annotations de type [texte][numéro]
    texte = re.sub(r'\[.*?\]\[\d+\]', '', texte)
    
    # Supprimer les espaces multiples et les lignes vides consécutives
    texte = re.sub(r'\n\s*\n', '\n', texte)
    texte = re.sub(r'[ ]{2,}', ' ', texte)
    
    # Supprimer les caractères spéciaux isolés comme '|'
    texte = re.sub(r'\|', '', texte)
    
    return texte.strip()

In [27]:
directive_UE_step_3 = nettoyer_markdown(directive_UE_step_2)
count_tokens(directive_UE_step_3) # gain de plus de 3k token

26144

- Step 1: gain de 13k token
- Step 2: gain de 8k/9K token
- Step 3: gain de 3k token

#### 3.2.2 - Reglement pipeline cleaning

In [252]:
# 
def supr_avant_reglement(texte):
    start_phrase = "ONT ADOPTÉ LE PRÉSENT RÈGLEMENT"
    index_debut = texte.find(start_phrase)
    if index_debut != -1:
        return texte[index_debut:]
    else:
        return "Le texte ne contient pas la phrase spécifiée."

In [255]:
directive_UE_CRR_step1 = supr_apres_directive(directive_UE_CRR)
count_tokens(directive_UE_CRR_step1)

373977

In [ ]:
def supr_apres_reglement(texte):
    end_phrase = "Par le Parlement européen"
    index_fin = texte.find(end_phrase)
    if index_fin != -1:
        return texte[:index_fin + len(end_phrase)]
    else:
        return texte  

In [263]:
directive_UE_CRR_step2= supr_apres_reglement(directive_UE_CRR_step1)

In [264]:
count_tokens(directive_UE_CRR_step2)

373973

#### 3.2.3 Snowball stemmer

In [277]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [282]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /Users/oussa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/oussa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [288]:
def preprocess_text_snowball(texte, langue='french'):
    """
    Cette fonction prend en entrée un texte, supprime les stopwords et applique le SnowballStemmer,
    puis retourne le texte prétraité sous forme de chaîne de caractères.
    
    :param texte: Le texte à traiter (chaîne de caractères).
    :param langue: La langue du texte et du stemmer (par défaut 'french').
    :return: Une chaîne de caractères du texte prétraité.
    """
    # Initialisation du stemmer pour la langue spécifiée
    stemmer = SnowballStemmer(langue)
    
    # Chargement des stopwords pour la langue spécifiée
    stop_words = set(stopwords.words(langue))
    
    # Tokenisation du texte
    mots = word_tokenize(texte, language=langue)
    
    # Filtrage des stopwords et racinisation des mots restants
    mots_filtrés_et_racinisés = [
        stemmer.stem(mot) for mot in mots
        if mot.isalnum() and mot.lower() not in stop_words
    ]
    
    # Conversion de la liste de mots en une chaîne de caractères
    texte_pretraite = ' '.join(mots_filtrés_et_racinisés)
    
    return texte_pretraite

In [295]:
directive_UE_CRR_stemed = preprocess_text_snowball(directive_UE_CRR_step2)
print("Nombre de token :",f"{count_tokens(directive_UE_CRR_stemed)}")


Nombre de token : 167041


### 4 - Long Context Question Answering (Option 1)

Pour le versionning de prompts dans Python, plusieurs outils possibles :
- PromptLayer
- LangSmith , 
- Agenta, 
- Mirascope et Lilypad
- prompt flow

In [29]:
fr_old_version= "Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7 , L. 632-13 et L. 632-16 du présent code sont tenues, nonobstant toutes dispositions contraires, de transmettre à des entreprises du même groupe ayant leur siège social dans l'un de ces Etats : 1° Les renseignements relatifs à leur situation financière nécessaires à l'organisation de la surveillance sur base consolidée et de la surveillance complémentaire de ces entités réglementées ou sociétés de financement ; 2° Les informations nécessaires à l'organisation de la lutte contre le blanchiment des capitaux et contre le financement du terrorisme ; 3° Les informations nécessaires à l'organisation de la détection des abus de marché mentionnés à l'article 16 du règlement (UE) n° 596/2014 du Parlement européen et du Conseil du 16 avril 2014 sur les abus de marché (règlement relatif aux abus de marché) et abrogeant la directive 2003/6/CE du Parlement européen et du Conseil et les directives 2003/124/CE, 2003/125/CE et 2004/72/CE de la Commission ; 4° Les informations nécessaires à la gestion des conflits d'intérêts au sens du 3° du I et 3° du II de l'article L. 533-10 . Ces dernières informations ne peuvent être communiquées à des personnes extérieures au groupe, à l'exception des autorités compétentes des Etats visés au premier alinéa. Cette exception ne s'étend pas aux autorités des Etats ou territoires dont la législation est reconnue insuffisante ou dont les pratiques sont considérées comme faisant obstacle à la lutte contre le blanchiment des capitaux ou le financement du terrorisme par l'instance internationale de concertation et de coordination en matière de lutte contre le blanchiment d'argent dont la liste est mise à jour par arrêté du ministre chargé de l'économie. Les personnes recevant ces informations sont tenues au secret professionnel dans les conditions et sous les peines mentionnées à l'article L. 511-33 , pour tous renseignements ou documents qu'elles seraient ainsi amenées à recevoir ou à détenir. Les dispositions du présent article ne font pas obstacle à l'application de la loi n° 78-17 du 6 janvier 1978 relative à l'informatique, aux fichiers et aux libertés."

In [30]:
fr_new_version= "Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, d'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7 , L. 632-13 et L. 632-16 du présent code sont tenues, nonobstant toutes dispositions contraires, de transmettre à des entreprises du même groupe ayant leur siège social dans l'un de ces Etats : 1° Les renseignements relatifs à leur situation financière nécessaires à l'organisation de la surveillance sur base consolidée et de la surveillance complémentaire de ces entités réglementées ou sociétés de financement ; 2° Les informations nécessaires à l'organisation de la lutte contre le blanchiment des capitaux et contre le financement du terrorisme ; 3° Les informations nécessaires à l'organisation de la détection des abus de marché mentionnés à l'article 16 du règlement (UE) n° 596/2014 du Parlement européen et du Conseil du 16 avril 2014 sur les abus de marché (règlement relatif aux abus de marché) et abrogeant la directive 2003/6/CE du Parlement européen et du Conseil et les directives 2003/124/CE, 2003/125/CE et 2004/72/CE de la Commission ; 4° Les informations nécessaires à la gestion des conflits d'intérêts au sens du 3° du I et 3° du II de l'article L. 533-10 . Ces dernières informations ne peuvent être communiquées à des personnes extérieures au groupe, à l'exception des autorités compétentes des Etats visés au premier alinéa. Cette exception ne s'étend pas aux autorités des Etats ou territoires dont la législation est reconnue insuffisante ou dont les pratiques sont considérées comme faisant obstacle à la lutte contre le blanchiment des capitaux ou le financement du terrorisme par l'instance internationale de concertation et de coordination en matière de lutte contre le blanchiment d'argent dont la liste est mise à jour par arrêté du ministre chargé de l'économie. Les personnes recevant ces informations sont tenues au secret professionnel dans les conditions et sous les peines mentionnées à l'article L. 511-33 , pour tous renseignements ou documents qu'elles seraient ainsi amenées à recevoir ou à détenir. Les dispositions du présent article ne font pas obstacle à l'application de la loi n° 78-17 du 6 janvier 1978 relative à l'informatique, aux fichiers et aux libertés."

In [31]:
output_buffer_4o = io.StringIO()

#### 4.1 - Version 1 - 3p 

Mise en place d'une fonction de récupération des textes européens avec un prompt a 3 paramètres:
- Le réglement UE 
- l'ancienne version du text
- la nouvelle version du text

In [ ]:
# LLM produit : analyse du changement avant et après et recherche de l article UE a l origine du changement.
prompt_LCQA_3p = f""" 
        Tu es un juriste expert avec au moins 10 ans d expérience.
        Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
        Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
        
        Ta tâche :
        -   Analyser les changements réglementaires dans la loi francaise avec pour objectif de trouver la ou les parties
        de la directive UE (article ou articles) a l origine de ce(s) changement(s).
        
        Régles : 
        - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
        - Tu feras ton analyse de la facon la plus conconcieuse et précise 
        - Tu ne pas pas reciter le contenu de la loi francaise avant et après modification
        
        Voici le text européen {directive_UE_step_3}
        Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
        Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
"""

In [33]:
for chunk in llm_4o.stream(prompt_LCQA_3p):
    print(chunk.content, end="", flush=True)

L'analyse des changements réglementaires dans la loi française, en lien avec la transposition de la directive européenne 2015/849, met en évidence les modifications apportées pour se conformer aux exigences de cette directive. Voici les points clés de la directive européenne qui ont conduit aux modifications dans la loi française :

1. **Extension du champ d'application** :
   - La directive européenne 2015/849, notamment dans ses articles 2 et 3, élargit le champ d'application des entités assujetties aux obligations de lutte contre le blanchiment de capitaux et le financement du terrorisme. Cela inclut les entreprises financières et les groupes comprenant des entités réglementées.
   - La loi française a été modifiée pour inclure les groupes au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2, en plus des groupes financiers et des conglomérats financiers.

2. **Obligations de transmission d'informations** :
   - L'a

In [34]:
def get_eu_data_3p(directive_UE, fr_old_version, fr_new_version):
    prompt_LCQA_3p = f""" 
        Tu es un juriste expert avec au moins 10 ans d expérience.
        Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
        Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
        
        Ta tâche :
        -   Analyser les changements réglementaires dans la loi francaise avec pour objectif de trouver la ou les parties
        de la directive UE (article ou articles) a l origine de ce(s) changement(s).
        
        Régles : 
        - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
        - Tu feras ton analyse de la facon la plus conconcieuse et précise 
        - Tu ne pas pas reciter le contenu de la loi francaise avant et après modification
        
        Voici le text européen {directive_UE}
        Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
        Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
"""
    for chunk in llm_4o.stream(prompt_LCQA_3p):
        print(chunk.content, end="", flush=True)

In [ ]:
get_eu_data_3p(directive_UE_step_3, fr_old_version, fr_new_version )

L'analyse des changements réglementaires dans la loi française, en lien avec la directive européenne 2015/849, révèle que les modifications apportées visent à renforcer les mesures de lutte contre le blanchiment de capitaux et le financement du terrorisme. Voici les points clés de la directive européenne qui ont influencé les modifications de la loi française :

1. **Extension du champ d'application** :
   - La directive européenne 2015/849, notamment dans ses articles 2 et 3, élargit le champ d'application des entités assujetties aux obligations de lutte contre le blanchiment de capitaux et le financement du terrorisme. Cela inclut les entreprises de financement et les entités réglementées.
   - La modification de la loi française inclut désormais les entités mentionnées au 1° bis ou 1° ter de l'article L. 561-2, ce qui correspond à l'élargissement du champ d'application prévu par la directive.

2. **Obligations de transmission d'informations** :
   - L'article 45 de la directive euro

#### 4.2 - Version 2 - 4p 

Rajout d'une fonction de dectection de changement entre les deux textes pour orienter l'analyse LLM

In [36]:
def comparer_phrases(phrase1, phrase2):
    mots1 = phrase1.split()
    mots2 = phrase2.split()
    sequence = difflib.SequenceMatcher(None, mots1, mots2)
    differences = []
    for tag, i1, i2, j1, j2 in sequence.get_opcodes():
        if tag == 'replace':
            differences.append(f"Remplacer '{' '.join(mots1[i1:i2])}' par '{' '.join(mots2[j1:j2])}'")
        elif tag == 'delete':
            differences.append(f"Supprimer '{' '.join(mots1[i1:i2])}'")
        elif tag == 'insert':
            differences.append(f"Insérer '{' '.join(mots2[j1:j2])}'")
    return differences

In [37]:
changement_derter= comparer_phrases(fr_old_version, fr_new_version)
print(changement_derter)

["Insérer 'd'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2,'"]


In [38]:
prompt_LCQA_4p = f""" 
        Tu es un juriste expert avec au moins 10 ans d expérience.
        Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
        Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
        
        
        Ta tâche :
        -   Analyser les changements réglementaires dans la loi francaise et de trouver la ou les parties
        de la directive UE (article ou articles) a l origine de ce changement réglementaire.
        
        Régles : 
        - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
        - Tu feras ton analyse de la facon la plus conconcieuse et précise 
        
        Voici le text européen {directive_UE}
        Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
        Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
        Voici une indication des changements entre les deux version du texte francais sur la quelle tu t appuieras pour
        trouver la partie du texte europeen a l origine de ce changement : {changement_derter}

"""

In [39]:
for chunk in llm_4o.stream(prompt_LCQA_4p):
    print(chunk.content, end="", flush=True)

Pour identifier la partie de la directive européenne (UE) 2015/849 qui est à l'origine du changement dans le texte de loi français, nous devons examiner les articles de la directive qui concernent les groupes financiers et la lutte contre le blanchiment des capitaux et le financement du terrorisme.

Le changement dans le texte français consiste à insérer la mention "d'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2,". Cela indique une extension du champ d'application de la loi française pour inclure des groupes financiers spécifiques et des entités réglementées.

En analysant la directive européenne, les articles pertinents sont ceux qui traitent des obligations des groupes financiers en matière de lutte contre le blanchiment des capitaux et le financement du terrorisme. Voici les articles de la directive européenne qui sont probablement à l'origine de ce changement :

1. **Article 45** de la directive 

In [40]:
def get_eu_data_4p(directive_UE, fr_old_version, fr_new_version):
    changement_derter= comparer_phrases(fr_old_version, fr_new_version)
    print(changement_derter)
    prompt_LCQA_4p = f""" 
            Tu es un juriste expert avec au moins 10 ans d expérience.
            Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
            Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
            
            
            Ta tâche :
            -   Analyser les changements réglementaires dans la loi francaise et de trouver la ou les parties
            de la directive UE (article ou articles) a l origine de ce changement réglementaire.
            
            Régles : 
            - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
            - Tu feras ton analyse de la facon la plus conconcieuse et précise 
            
            Voici le text européen {directive_UE}
            Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
            Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
            Voici une indication des changements entre les deux version du texte francais sur la quelle tu t appuieras pour
            trouver la partie du texte europeen a l origine de ce changement : {changement_derter}

    """
    for chunk in llm_4o.stream(prompt_LCQA_4p):
        print(chunk.content, end="", flush=True)
        
                    
        

In [235]:
count_tokens(directive_UE_step_3)

26144

In [ ]:
get_eu_data_4p(directive_UE_step_3, fr_old_version, fr_new_version)
# RUN 1: article 2 + article 45 + article 9 + articles 37 et 41
# RUN 2: article 46 + article 45 + 45 paragraphe 8 + article 9 + articles 39
# RUN 3: l'article 45 de la directive (UE) 2015/849.

["Insérer 'd'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2,'"]
Pour analyser les changements réglementaires dans la loi française et identifier la ou les parties de la directive européenne (article ou articles) à l'origine de ce changement, nous devons examiner les modifications apportées et les comparer avec les exigences de la directive (UE) 2015/849.

### Analyse des Changements

**Texte français avant l'entrée en vigueur du texte européen :**
Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à 

#### 4.2 - Testing on some Code monétaire et financier articles 

##### 4.2.1 - Exemple 1 : L561-2 du cmf (prompt v1 - 3p)

In [42]:
fr_old_version="Sont assujettis aux obligations prévues par les dispositions des sections 2 à 7 du présent chapitre : 1° Les organismes, institutions et services régis par les dispositions du titre Ier du présent livre, y compris les succursales des établissements de crédit mentionnés à l'article L. 511-22 et des établissements financiers mentionnés à l'article L. 511-23 ; 1° bis Les établissements de paiement régis par les dispositions du chapitre II du titre II du présent livre y compris les succursales des établissements de paiement mentionnés au II de l'article L. 522-13 ; 1° ter Les établissements de monnaie électronique régis par le chapitre VI du titre II du présent livre y compris les succursales des établissements de monnaie électronique mentionnés à l'article L. 526-24 ; 1° quater Les établissements de crédit, les établissements de paiement, les établissements de monnaie électronique et les prestataires de services sur crypto-actifs ayant leur siège social dans un autre Etat membre de l'Union européenne ou partie à l'Espace économique européen en tant qu'ils exercent leur activité sur le territoire national en ayant recours aux services d'un ou plusieurs agents pour la fourniture de services de paiement en France ou d'une ou plusieurs personnes en vue de distribuer en France de la monnaie électronique et les prestataires de services sur crypto-actifs au sens de l'article L. 525-8 ou d'une ou plusieurs personnes en vue de la fourniture de services sur cryptoactifs ; 2° Les entreprises mentionnées aux articles L. 310-1 et L. 310-2 du code des assurances ; 2° bis Les institutions ou unions régies par le titre III du livre IX du code de la sécurité sociale ; 2° ter Les mutuelles et unions réalisant des opérations mentionnées au 1° du I de l'article L. 111-1 du code de la mutualité ; 2° quater Les fonds de retraite professionnelle supplémentaire mentionnés à l'article L. 381-1 du code des assurances ; 2° quinquies Les mutuelles ou unions de retraite professionnelle supplémentaire mentionnées à l'article L. 214-1 du code de la mutualité ; 2° sexies Les institutions de retraite professionnelle supplémentaire mentionnées à l'article L. 942-1 du code de la sécurité sociale ; 3° Les intermédiaires en opérations de banque et en services de paiement mentionnés à l'article L. 519-1 lorsqu'ils agissent en vertu d'un mandat délivré par un client et qu'ils se voient confier des fonds en tant que mandataire des parties ; 3° bis Les intermédiaires d'assurance définis à l'article L. 511-1 du code des assurances sauf ceux qui agissent sous l'entière responsabilité de l'organisme ou du courtier d'assurance ; 4° Les intermédiaires en financement participatif mentionnés à l'article L. 548-2 ; 5° La Banque de France, l'institut d'émission des départements d'outre-mer mentionné à l'article L. 721-7 du présent code et l'institut d'émission d'outre-mer mentionné à l'article L. 721-18 et L. 721-19 du même code ; 6° Les entreprises d'investissement, y compris les succursales d'entreprises d'investissement mentionnées à l'article L. 532-18-1 ainsi que les succursales d'entreprises d'investissement mentionnées à l'article L. 532-48 les personnes mentionnées à l'article L. 440-2, les entreprises de marché mentionnées à l'article L. 421-2, les dépositaires centraux mentionnés aux 1° et 2° du I de l'article L. 441-1 et gestionnaires de systèmes de règlement et de livraison d'instruments financiers, les conseillers en investissements financiers, les prestataires de services de financement participatif au titre de leurs activités mentionnées à l'article L. 547-4 et les intermédiaires habilités mentionnés à l'article L. 211-4, ainsi que les placements collectifs mentionnés au I de l'article L. 214-1 et les sociétés de gestion de placements collectifs mentionnées à l'article L. 543-1 et les succursales des sociétés de gestion européennes d'OPCVM et de FIA mentionnées aux articles L. 532-20-1 et L. 532-21-3 ; 6° bis Les prestataires de services d'investissement ayant leur siège social dans un autre Etat membre de l'Union européenne en tant qu'ils exercent leur activité sur le territoire national en ayant recours à des agents liés mentionnés à l'article L. 545-1 du code monétaire et financier ; 7° Les changeurs manuels ; 7° bis a) Les prestataires des services mentionnés aux 1° à 4° de l'article L. 54-10-2, et ; b) Les prestataires des services autorisés à fournir des services sur crypto-actifs conformément à l'article 59 du règlement (UE) 2023/1114 du Parlement européen et du Conseil du 31 mai 2023 sur les marchés de crypto-actifs et modifiant les règlements (UE) n° 1093/2010 et (UE) n° 1095/2010 et les directives 2013/36/ UE et (UE) 2019/1937, y compris les succursales établies en France des prestataires ayant leur siège social dans un autre Etat membre de l'Union européenne ou partie à l'Espace économique européen à l'exception de ceux mentionnés au b du 7° quater du présent article ; 7° ter (Abrogé) ; 7° quater a) Les prestataires agréés au titre de l'article L. 54-10-5, à l'exception des prestataires mentionnés au a du 7° bis du présent article ; et b) Les conseillers en investissements financiers, dépositaires centraux et sociétés de gestion de placements collectifs mentionnées à l'article L. 543-1 autorisés à fournir des services sur crypto-actifs conformément à l'article 59 du règlement (UE) 2023/1114 du Parlement européen et du Conseil du 31 mai 2023 sur les marchés de crypto actifs et modifiant les règlements (UE) n° 1093/2010 et (UE) n° 1095/2010 et les directives 2013/36/ UE et (UE) 2019/1937, y compris les succursales établies en France des sociétés de gestion européennes d'OPCVM et de FIA mentionnées aux articles L. 532-20-1 et L. 532-21-3 ayant leur siège social dans un autre Etat membre de l'Union européenne ou partie à l'Espace économique européen ou les prestataires de services autorisés uniquement pour fournir les services de gestion de portefeuille de crypto-actifs ou de conseils en crypto-actifs mentionnés respectivement au i et au h du point 16 du paragraphe 1 de l'article 3 du même règlement ; 8° Les personnes exerçant les activités mentionnées au 1°, mais concernant leur activité de location uniquement en exécution d'un mandat de transaction de biens immeubles dont le loyer mensuel est supérieur ou égal à 10 000 euros, ainsi qu'aux 2°, 4°, 5° et 8° de l'article 1er de la loi n° 70-9 du 2 janvier 1970 réglementant les conditions d'exercice des activités relatives à certaines opérations portant sur les immeubles et les fonds de commerce ; 9° Les opérateurs de jeux ou de paris autorisés sur le fondement de l'article L. 321-1 et L. 321-3 du code de la sécurité intérieure, du V de l'article 34 de la loi n° 2017-257 du 28 février 2017 relative au statut de Paris et à l'aménagement métropolitain et leurs représentants légaux et directeurs responsables ; 9° bis Les opérateurs de jeux ou de paris autorisés sur le fondement de l'article 21 de la loi n° 2010-476 du 12 mai 2010 relative à l'ouverture à la concurrence et à la régulation du secteur des jeux d'argent et de hasard en ligne et leurs représentants légaux, de l'article 5 de la loi du 2 juin 1891, ayant pour objet de réglementer l'autorisation et le fonctionnement des courses de chevaux, de l'article 137 de la loi n° 2019-486 du 22 mai 2019 relative à la croissance et la transformation des entreprises ; 10° Les personnes qui négocient des œuvres d'art et des antiquités ou agissent en qualité d'intermédiaires dans le commerce des œuvres d'art et d'antiquités, y compris lorsque celui-ci est réalisé par des galeries d'art, lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros et les personnes qui entreposent ou négocient des œuvres d'art ou agissent en qualité d'intermédiaires dans le commerce des œuvres d'art quand celui-ci est réalisé dans des ports francs ou zones franches, lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros ; 11° Les personnes acceptant des paiements en espèces ou au moyen de monnaie électronique d'un montant supérieur à un seuil fixé par décret et se livrant au commerce de biens ; 11° bis Les personnes, autres que celles mentionnées aux 1° à 7° quater, se livrant à titre habituel et principal au commerce de métaux précieux ou de pierres précieuses, lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros ; 12° Les experts-comptables, les salariés autorisés à exercer la profession d'expert-comptable en application des articles 83 ter et 83 quater de l'ordonnance n° 45-2138 du 19 septembre 1945 portant institution de l'ordre des experts-comptables et réglementant les titres et la profession d'expert-comptable ; 12° bis Les commissaires aux comptes ; 13° Les avocats au Conseil d'Etat et à la Cour de cassation, les avocats, les notaires, les commissaires de justice, les administrateurs judiciaires et les mandataires judiciaires, dans les conditions prévues à l'article L. 561-3 ; 14° Les opérateurs de ventes volontaires de meubles aux enchères publiques lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros ; 15° Les personnes exerçant l'activité de domiciliation mentionnée aux articles L. 123-11-2 et suivants du code de commerce ; 16° Les personnes exerçant l'activité d'agents sportifs mentionnés à l'article L. 222-7 du code du sport ; 17° Les personnes autorisées au titre du I de l'article L. 621-18-5 ; 18° Les caisses des règlements pécuniaires des avocats créées en application du 9° de l'article 53 de la loi n° 71-1130 du 31 décembre 1971 au titre des fonds, effets ou valeurs déposés par les avocats pour le compte de leurs clients dans le cadre des activités mentionnées au I de l'article L. 561-3 ; 19° Les greffiers des tribunaux de commerce mentionnés à l'article L. 741-1 du code de commerce ; 20° Les gestionnaires de crédits. Les personnes assujetties mentionnées aux 1° à 20° comprennent les personnes physiques et les personnes morales."

In [43]:
fr_new_version="Sont assujettis aux obligations prévues par les dispositions des sections 2 à 7 du présent chapitre : 1° Les organismes, institutions et services régis par les dispositions du titre Ier du présent livre, y compris les succursales des établissements de crédit mentionnés à l'article L. 511-22 et des établissements financiers mentionnés à l'article L. 511-23 ; 1° bis Les établissements de paiement régis par les dispositions du chapitre II du titre II du présent livre y compris les succursales des établissements de paiement mentionnés au II de l'article L. 522-13 ; 1° ter Les établissements de monnaie électronique régis par le chapitre VI du titre II du présent livre y compris les succursales des établissements de monnaie électronique mentionnés à l'article L. 526-24 ; 1° quater Les établissements de crédit, les établissements de paiement et les établissements de monnaie électronique ayant leur siège social dans un autre Etat membre de l'Union européenne ou partie à l'Espace économique européen en tant qu'ils exercent leur activité sur le territoire national en ayant recours aux services d'un ou plusieurs agents pour la fourniture de services de paiement en France ou d'une ou plusieurs personnes en vue de distribuer en France de la monnaie électronique au sens de l'article L. 525-8 ; 2° Les entreprises mentionnées aux articles L. 310-1 et L. 310-2 du code des assurances ; 2° bis Les institutions ou unions régies par le titre III du livre IX du code de la sécurité sociale ; 2° ter Les mutuelles et unions réalisant des opérations mentionnées au 1° du I de l'article L. 111-1 du code de la mutualité ; 2° quater Les fonds de retraite professionnelle supplémentaire mentionnés à l'article L. 381-1 du code des assurances ; 2° quinquies Les mutuelles ou unions de retraite professionnelle supplémentaire mentionnées à l'article L. 214-1 du code de la mutualité ; 2° sexies Les institutions de retraite professionnelle supplémentaire mentionnées à l'article L. 942-1 du code de la sécurité sociale ; 3° Les intermédiaires en opérations de banque et en services de paiement mentionnés à l'article L. 519-1 lorsqu'ils agissent en vertu d'un mandat délivré par un client et qu'ils se voient confier des fonds en tant que mandataire des parties ; 3° bis Les intermédiaires d'assurance définis à l' article L. 511-1 du code des assurances sauf ceux qui agissent sous l'entière responsabilité de l'organisme ou du courtier d'assurance ; 4° Les intermédiaires en financement participatif mentionnés à l'article L. 548-2 ; 5° La Banque de France, l'institut d'émission des départements d'outre-mer mentionné à l'article L. 711-2 du présent code et l'institut d'émission d'outre-mer mentionné à l'article L. 712-4 du même code ; 6° Les entreprises d'investissement, y compris les succursales d'entreprises d'investissement mentionnées à l'article L. 532-18-1 ainsi que les succursales d'entreprises d'investissement mentionnées à l'article L. 532-48, les personnes mentionnées à l'article L. 440-2 , les entreprises de marché mentionnées à l'article L. 421-2 , les dépositaires centraux et gestionnaires de systèmes de règlement et de livraison d'instruments financiers, les conseillers en investissements financiers, les conseillers en investissements participatifs et les intermédiaires habilités mentionnés à l'article L. 211-4 , ainsi que les placements collectifs mentionnés au I de l'article L. 214-1 et les sociétés de gestion de placements collectifs mentionnées à l'article L. 543-1 et les succursales des sociétés de gestion européennes d'OPCVM et de FIA mentionnées aux articles L. 532-20-1 et L. 532-21-3 ; 6° bis Les prestataires de services d'investissement ayant leur siège social dans un autre Etat membre de l'Union européenne en tant qu'ils exercent leur activité sur le territoire national en ayant recours à des agents liés mentionnés à l' article L. 545-1 du code monétaire et financier ; 7° Les changeurs manuels ; 7° bis Les prestataires des services mentionnés aux 1° et 2° de l'article L. 54-10-2 ; 7° ter Les émetteurs de jetons ayant obtenu le visa mentionné à l'article L. 552-4 dans le cadre de l'offre ayant fait l'objet du visa et dans la limite des transactions avec les souscripteurs prenant part à cette offre ; 7° quater Les prestataires agréés au titre de l'article L. 54-10-5, à l'exception des prestataires mentionnés au 7° bis du présent article ; 8° Les personnes exerçant les activités mentionnées aux 1°, uniquement en ce qui concerne leur activité de location en exécution d'un mandat de transaction de biens immeubles dont le loyer mensuel est supérieur ou égal à 10 000 euros, 2°, 4°, 5° et 8° de l'article 1er de la loi n° 70-9 du 2 janvier 1970 réglementant les conditions d'exercice des activités relatives à certaines opérations portant sur les immeubles et les fonds de commerce ; 9° Les opérateurs de jeux ou de paris autorisés sur le fondement de l'article L. 321-1 et L. 321-3 du code de la sécurité intérieure, du V de l'article 34 de la loi n° 2017-257 du 28 février 2017 relative au statut de Paris et à l'aménagement métropolitain et leurs représentants légaux et directeurs responsables ; 9° bis Les opérateurs de jeux ou de paris autorisés sur le fondement de l'article 21 de la loi n° 2010-476 du 12 mai 2010 relative à l'ouverture à la concurrence et à la régulation du secteur des jeux d'argent et de hasard en ligne et leurs représentants légaux, de l'article 5 de la loi du 2 juin 1891, ayant pour objet de réglementer l'autorisation et le fonctionnement des courses de chevaux, de l'article 137 de la loi n° 2019-486 du 22 mai 2019 relative à la croissance et la transformation des entreprises ; 10° Les personnes qui négocient des œuvres d'art et des antiquités ou agissent en qualité d'intermédiaires dans le commerce des œuvres d'art et d'antiquités, y compris lorsque celui-ci est réalisé par des galeries d'art, lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros et les personnes qui entreposent ou négocient des œuvres d'art ou agissent en qualité d'intermédiaires dans le commerce des œuvres d'art quand celui-ci est réalisé dans des ports francs ou zones franches, lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros ; 11° Les personnes acceptant des paiements en espèces ou au moyen de monnaie électronique d'un montant supérieur à un seuil fixé par décret et se livrant au commerce de biens ; 12° Les experts-comptables, les salariés autorisés à exercer la profession d'expert-comptable en application des articles 83 ter et 83 quater de l'ordonnance n° 45-2138 du 19 septembre 1945 portant institution de l'ordre des experts-comptables et réglementant les titres et la profession d'expert-comptable ; 12° bis Les commissaires aux comptes ; 13° Les avocats au Conseil d'Etat et à la Cour de cassation, les avocats, les notaires, les huissiers de justice, les administrateurs judiciaires, les mandataires judiciaires et les commissaires-priseurs judiciaires, dans les conditions prévues à l'article L. 561-3 ; 14° Les opérateurs de ventes volontaires de meubles aux enchères publiques lorsque la valeur de la transaction ou d'une série de transactions liées est d'un montant égal ou supérieur à 10 000 euros ; 15° Les personnes exerçant l'activité de domiciliation mentionnée aux articles L. 123-11-2 et suivants du code de commerce ; 16° Les personnes exerçant l'activité d'agents sportifs mentionnés à l' article L. 222-7 du code du sport ; 17° Les personnes autorisées au titre du I de l'article L. 621-18-5 ; 18° Les caisses des règlements pécuniaires des avocats créées en application du 9° de l'article 53 de la loi n° 71-1130 du 31 décembre 1971 au titre des fonds, effets ou valeurs déposés par les avocats pour le compte de leurs clients dans le cadre des activités mentionnées au I de l'article L. 561-3 ; 19° Les greffiers des tribunaux de commerce mentionnés à l'article L. 741-1 du code de commerce. Les personnes assujetties mentionnées aux 1° à 19° comprennent les personnes physiques et les personnes morales."

In [ ]:
get_eu_data_3p(directive_UE_step_3, fr_old_version, fr_new_version )

Pour analyser les changements réglementaires dans la loi française et identifier les parties de la directive européenne (articles) à l'origine de ces changements, nous devons comparer les versions avant et après l'entrée en vigueur de la directive européenne 2015/849. Voici les principaux changements et les articles de la directive européenne correspondants :

1. **Ajout des prestataires de services sur crypto-actifs** :
   - **Avant** : Les prestataires de services sur crypto-actifs n'étaient pas spécifiquement mentionnés.
   - **Après** : Les prestataires de services sur crypto-actifs sont inclus dans les catégories assujetties aux obligations.
   - **Directive européenne** : Article 2(1)(3)(a) et (b) de la directive 2015/849, qui inclut les prestataires de services sur crypto-actifs dans les entités assujetties.

2. **Modification des seuils pour les transactions en espèces** :
   - **Avant** : Les personnes acceptant des paiements en espèces ou au moyen de monnaie électronique d'un

##### 4.2.2 - Exemple 1 : L561-2 du cmf (prompt v2 - 4p)

In [ ]:
get_eu_data_4p(directive_UE_step_3, fr_old_version, fr_new_version)

["Remplacer 'paiement,' par 'paiement et'", "Supprimer 'et les prestataires de services sur crypto-actifs'", "Supprimer 'et les prestataires de services sur crypto-actifs'", "Supprimer 'ou d'une ou plusieurs personnes en vue de la fourniture de services sur cryptoactifs'", "Remplacer 'l'article' par 'l' article'", "Remplacer '721-7' par '711-2'", "Remplacer '721-18 et L. 721-19' par '712-4'", "Remplacer '532-48' par '532-48,'", "Remplacer '440-2,' par '440-2 ,'", "Remplacer '421-2,' par '421-2 ,'", "Supprimer 'mentionnés aux 1° et 2° du I de l'article L. 441-1'", "Remplacer 'prestataires de services de financement participatif au titre de leurs activités mentionnées à l'article L. 547-4' par 'conseillers en investissements participatifs'", "Remplacer '211-4,' par '211-4 ,'", "Remplacer 'l'article' par 'l' article'", "Supprimer 'a)'", "Remplacer 'à 4° de l'article L. 54-10-2, et ; b) Les prestataires des services autorisés à fournir des services sur crypto-actifs conformément à l'articl

##### 4.2.2 - Exemple 2 : L561-10 du cmf (prompt v1 - 3p)

In [46]:
fr_old_version="Les personnes mentionnées à l'article L. 561-2 appliquent des mesures de vigilance complémentaires à l'égard de leur client, en sus des mesures prévues aux articles L. 561-5 et L. 561-5-1 , lorsque : 1° Le client ou son représentant légal n'est pas physiquement présent aux fins de l'identification au moment de l'établissement de la relation d'affaires ; 2° Le client, le cas échéant son bénéficiaire effectif, le bénéficiaire d'un contrat d'assurance-vie ou de capitalisation, le cas échéant son bénéficiaire effectif, est une personne qui est exposée à des risques particuliers en raison des fonctions politiques, juridictionnelles ou administratives qu'elle exerce ou a exercées pour le compte d'un Etat ou de celles qu'exercent ou ont exercées des membres directs de sa famille ou des personnes connues pour lui être étroitement associées ou le devient en cours de relation d'affaires ; 3° Le produit ou l'opération présente, par sa nature, un risque particulier de blanchiment de capitaux ou de financement du terrorisme, notamment lorsqu'ils favorisent l'anonymat ; 4° L'opération est une opération pour compte propre ou pour compte de tiers effectuée avec des personnes physiques ou morales, y compris leurs filiales ou établissements, domiciliées, enregistrées ou établies dans un Etat ou un territoire figurant sur les listes publiées par le Groupe d'action financière parmi ceux dont la législation ou les pratiques font obstacle à la lutte contre le blanchiment des capitaux et le financement du terrorisme ou par la Commission européenne en application de l'article 9 de la directive (UE) 2015/849 du 20 mai 2015 relative à la prévention de l'utilisation du système financier aux fins du blanchiment de capitaux ou du financement du terrorisme. S'il n'existe pas de soupçon de blanchiment des capitaux ou de financement du terrorisme, les personnes mentionnées à l'article L. 561-2 peuvent ne pas appliquer aux clients mentionnés aux 1° et 2° les mesures de vigilance complémentaires prévues par le présent article lorsque la relation d'affaires est établie avec une personne mentionnée au 2° de l'article L. 561-9 ou est établie exclusivement pour un ou plusieurs produits mentionnés au même 2° de l'article L. 561-9 . Un décret en Conseil d'Etat précise les catégories de personnes mentionnées au 2°, la liste des produits et des opérations mentionnées au 3°, ainsi que les mesures de vigilance complémentaires."

In [47]:
fr_new_version="Les personnes mentionnées à l'article L. 561-2 appliquent des mesures de vigilance complémentaires à l'égard de leur client, en sus des mesures prévues aux articles L. 561-5 et L. 561-5-1 , lorsque : 1° Le client, le cas échéant son bénéficiaire effectif, le bénéficiaire d'un contrat d'assurance-vie ou de capitalisation, le cas échéant son bénéficiaire effectif, est une personne qui est exposée à des risques particuliers en raison des fonctions politiques, juridictionnelles ou administratives qu'elle exerce ou a exercées ou de celles qu'exercent ou ont exercées des membres directs de sa famille ou des personnes connues pour lui être étroitement associées ou le devient en cours de relation d'affaires ; 2° Le produit ou l'opération présente, par sa nature, un risque particulier de blanchiment de capitaux ou de financement du terrorisme, notamment lorsqu'ils favorisent l'anonymat ; 3° L'opération est une opération pour compte propre ou pour compte de tiers effectuée avec des personnes physiques ou morales, y compris leurs filiales ou établissements ou toute autre entité, domiciliées, enregistrées ou établies dans un Etat ou un territoire figurant sur les listes publiées par le Groupe d'action financière parmi ceux dont la législation ou les pratiques font obstacle à la lutte contre le blanchiment des capitaux et le financement du terrorisme ou par la Commission européenne en application de l'article 9 de la directive (UE) 2015/849 du 20 mai 2015 relative à la prévention de l'utilisation du système financier aux fins du blanchiment de capitaux ou du financement du terrorisme. S'il n'existe pas de soupçon de blanchiment des capitaux ou de financement du terrorisme, les personnes mentionnées à l'article L. 561-2 peuvent ne pas appliquer aux clients mentionnés au 1° les mesures de vigilance complémentaires prévues par le présent article lorsque la relation d'affaires est établie avec une personne mentionnée au 2° de l'article L. 561-9 ou est établie exclusivement pour un ou plusieurs produits mentionnés au même 2° de l'article L. 561-9 . Un décret en Conseil d'Etat précise les catégories de personnes mentionnées au 1°, la liste des produits et des opérations mentionnées au 2°, ainsi que les mesures de vigilance complémentaires."

In [ ]:
get_eu_data_3p(directive_UE_step_3, fr_old_version, fr_new_version )

L'analyse des changements réglementaires dans la loi française, en lien avec la directive (UE) 2015/849, révèle plusieurs points de correspondance et d'influence directe de la directive sur la législation nationale. Voici les principaux articles de la directive européenne qui ont conduit aux modifications observées dans la loi française :

1. **Article 18 de la directive (UE) 2015/849** :
   - Cet article impose des mesures de vigilance renforcées à l'égard de la clientèle dans des situations de risque plus élevé, notamment pour les personnes politiquement exposées (PEP). La loi française a intégré cette exigence en précisant que les mesures de vigilance complémentaires s'appliquent aux clients exposés à des risques particuliers en raison de leurs fonctions politiques, juridictionnelles ou administratives, ainsi qu'aux membres de leur famille et aux personnes étroitement associées.

2. **Article 20 de la directive (UE) 2015/849** :
   - Cet article détaille les mesures spécifiques à ap

In [49]:
for chunk in llm_4o.stream(prompt_LCQA_3p):
    print(chunk.content, end="", flush=True)

L'analyse des changements réglementaires dans la loi française, en lien avec la directive européenne 2015/849, révèle que les modifications apportées visent à renforcer les mesures de lutte contre le blanchiment de capitaux et le financement du terrorisme. Voici les points clés de la directive européenne qui ont influencé les modifications de la loi française :

1. **Extension du champ d'application** :
   - La directive européenne 2015/849, notamment dans ses articles 2 et 3, élargit le champ d'application des entités assujetties aux obligations de lutte contre le blanchiment de capitaux et le financement du terrorisme. Cela inclut les entreprises de financement et les entités réglementées.
   - La loi française a été modifiée pour inclure explicitement les entités mentionnées au 1° bis ou 1° ter de l'article L. 561-2, conformément à l'article 2 de la directive.

2. **Obligations de transmission d'informations** :
   - L'article 45 de la directive impose aux entités assujetties de met

##### 4.2.3 - Exemple 2 : L561-10 du cmf (prompt v2 - 4p)

In [ ]:
get_eu_data_4p(directive_UE_step_3, fr_old_version, fr_new_version)

["Supprimer 'client ou son représentant légal n'est pas physiquement présent aux fins de l'identification au moment de l'établissement de la relation d'affaires ; 2° Le'", "Supprimer 'pour le compte d'un Etat'", "Remplacer '3°' par '2°'", "Remplacer '4°' par '3°'", "Remplacer 'établissements,' par 'établissements ou toute autre entité,'", "Remplacer 'aux' par 'au'", "Supprimer 'et 2°'", "Remplacer '2°,' par '1°,'", "Remplacer '3°,' par '2°,'"]
Pour analyser les changements réglementaires dans la loi française et identifier les parties de la directive européenne (UE) 2015/849 à l'origine de ces changements, nous allons examiner les modifications apportées au texte français et les comparer avec les dispositions de la directive européenne.

### Changements dans la loi française

1. **Suppression de la mention du représentant légal non physiquement présent** :
   - **Avant** : "Le client ou son représentant légal n'est pas physiquement présent aux fins de l'identification au moment de l'ét

### 5 - Retrieval Augmented Generation (Option 2)

#### 5.1 - Chunking

##### 5.1.1 - Option 1: RecursiveCharacterTextSplitter  v1 : directive_UE_step_3

Essaye de découper le texte avec le séparateur le plus large (par exemple un double saut de ligne), puis s’il n’arrive pas à produire un chunk qui respecte les contraintes (comme la taille maximale si elle est spécifiée), il descend au séparateur suivant (par exemple un point, puis un espace).

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

In [ ]:
# On splitte par article grâce à un séparateur explicite
article_splitter = RecursiveCharacterTextSplitter(
    separators=   [ "le"],
    chunk_overlap=0)


In [228]:
# Découpage en chunks
chunks = article_splitter.split_text(directive_UE_step_3)
chunks

["ONT ADOPTÉ LA PRÉSENTE DIRECTIVE:\nCHAPITRE I\nDISPOSITIONS GÉNÉRALES\nSECTION\xa01\nObjet, champ d'application et définitions\nArtic",
 "le premier\n1.\xa0\xa0\xa0La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme.\n2.\xa0\xa0\xa0Les États membres veil",
 'lent à ce que le blanchiment de capitaux et',
 'le financement du terrorisme soient interdits.\n3.\xa0\xa0\xa0Aux fins de la présente directive, sont considérés comme blanchiment de capitaux ',
 'les\xa0agissements ci-après énumérés, commis intentionnellement:\n la conversion ou',
 "le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminel",
 "le ou d'une participation à une activité criminelle, dans le but de dissimu",
 "ler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une tel",
 "le activité à échapper aux\xa0conséquences juridiques des actes qu'elle

In [148]:
# On remet "Article " devant chaque chunk (car le séparateur les enlève)
chunks = ["Article " + chunk.strip() if not chunk.startswith("Article") else chunk.strip() for chunk in chunks]
chunks

["Article ONT ADOPTÉ LA PRÉSENTE DIRECTIVE:\nCHAPITRE I\nDISPOSITIONS GÉNÉRALES\nSECTION\xa01\nObjet, champ d'application et définitions\nArticle premier\n1.\xa0\xa0\xa0La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme.\n2.\xa0\xa0\xa0Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits.\n3.\xa0\xa0\xa0Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les\xa0agissements ci-après énumérés, commis intentionnellement:\n la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux\xa0conséquences juridiques des actes qu'elle a commis; \n le fait de dissi

In [123]:
# Affichage des chunks
for idx, chunk in enumerate(chunks):
    print(f"--- Chunk {idx+1} ---\n{chunk}\n")

--- Chunk 1 ---
Article ONT ADOPTÉ LA PRÉSENTE DIRECTIVE:
CHAPITRE I
DISPOSITIONS GÉNÉRALES
SECTION 1
Objet, champ d'application et définitions
Article premier
1.   La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme.
2.   Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits.
3.   Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les agissements ci-après énumérés, commis intentionnellement:
 la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux conséquences juridiques des actes qu'elle a commis; 
 le fait de dissimuler ou de déguiser la nature, 

##### 5.1.1 - Option 2: MarkdownHeaderTextSplitter

In [105]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

#### 5.2 - Test of  structure extraction with LLM (Art level) 

- PromptLayer
	•	Interface visuelle pour éditer et déployer des prompts sans code

	•	Contrôle de version à l’échelle de l’entreprise avec historique détaillé

	•	Tableau de bord analytique pour suivre les performances

	•	Tests A/B intégrés pour évaluer les prompts

	•	Support multi-modèles avec système de template unifié

- Agenta
	•	Simplifie les tests de prompts avec contrôle de version

	•	Permet des comparaisons côte à côte entre différents LLM

	•	Plateforme open source pour le développement d’applications LLM

- LangSmith
	•	Hub centralisé (LangChain Hub) pour archiver et versionner les prompts

	•	Intégration avec LangChain pour gérer les chaînes et agents intelligents

	•	Commandes pour télécharger des prompts spécifiques avec leur hash de version

- Mirascope et Lilypad
	•	Toolkit Python léger pour construire des applications LLM de qualité production

	•	Système de gestion et versionnage de prompts intégré (Lilypad)

	•	Permet l’utilisation locale avec vos prompts LLM

	•	Regroupe tous les éléments affectant la qualité d’un appel LLM en une unité cohésive


- Pezzo
	•	Plateforme LLMOps open source pour l’ingénierie de prompts

	•	Interface graphique pour créer, tester et versionner les prompts
	
	•	Utilise des hashes SHA comme numéros de version

In [ ]:
prompt_art_extract = f"""
        You are an expert data scientist and data engineer with an expertise in law .
        That means that you have a deep knowlage of law.
        
        To do: 
          - Extract the structure of this document to prepare to make the best and most reliable embedding.
          
        Rules: 
          - Be as acurate as possible.
          - Do not ommit information.
          - Do not omit the structure under "Article"
          - Do not translate the text : 
          {UE_text} 
          """

In [ ]:
for chunk in llm_4o.stream(prompt_art_extract):
    print(chunk.content, end="", flush=True)

Pour préparer la structure de ce document afin de créer les meilleurs et les plus fiables embeddings, voici une extraction détaillée de la structure du document, en respectant les règles données :

1. **En-tête du document**
   - Date : 5.6.2015
   - Langue : FR
   - Source : Journal officiel de l'Union européenne
   - Référence : L 141/73

2. **Titre du document**
   - DIRECTIVE (UE) 2015/849 DU PARLEMENT EUROPÉEN ET DU CONSEIL du 20 mai 2015

3. **Objet de la directive**
   - Relative à la prévention de l'utilisation du système financier aux fins du blanchiment de capitaux ou du financement du terrorisme, modifiant le règlement (UE) no 648/2012 du Parlement européen et du Conseil et abrogeant la directive 2005/60/CE du Parlement européen et du Conseil et la directive 2006/70/CE de la Commission

4. **Préambule**
   - Introduction
   - Vu le traité sur le fonctionnement de l'Union européenne, et notamment son article 114,
   - Vu la proposition de la Commission européenne,
   - Après 

#### 5.4 - Extract structure with LLM (Whole structure)

#### 5.4.1 - With Gpt 4o

In [ ]:
# LLM produit : analyse du changement avant et après et recherche de l article UE a l origine du changement.
prompt_structure_extract_json = f""" 
        You are an expert data scientist and data engineer with an expertise in law .
        That means that you have a deep knowlage of law.
        
        To do: 
          - Extract the whole structure of this document to prepare to make the best and most reliable embedding.
          - The output is a tabular Json. 
          Example de contenu a mettre en accollade : 
           'article 1er', 'paragraphe n°3', 'contenu': "3. Aux fins de la présente directive, sont 
          criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ...".
          
        Rules: 
          - Be as acurate as possible.
          - Do not ommit information.
          - You will be starting from the "Article"
          - Do not omit the structure under "Article" until the  "Article" : Article 69 Les États membres sont destinataires de la présente directive.
          - Do not translate the text : {UE_text} 
          
"""

In [ ]:
for chunk in llm_4o.stream(prompt_structure_extract_json):
    print(chunk.content, end="", flush=True)  # Affichage en temps réel
    output_buffer_4o.write(chunk.content)  # Stockage dans le buffer

# Récup
result_text_extract_4o = output_buffer_4o.getvalue()

```json
[
  {
    "article": "Article premier",
    "paragraphe n°1": "1. La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme.",
    "paragraphe n°2": "2. Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits.",
    "paragraphe n°3": "3. Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les agissements ci-après énumérés, commis intentionnellement:",
    "paragraphe n°3, point a": "a) la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux conséquences juridiques des actes qu'elle a commis;",
    "paragraphe n°3, point b": "b) le fait de dissimuler 

In [ ]:
count_tokens(result_text_extract_4o)

1339

- Problem of reaching max token 4K for GPT 4o.
- Let's use 4o mini for higher context windows, lower cost for this low complicated tasked that do not require a deep undersating of the content.

#### 5.4.2 - With Gpt 4o mini

In [ ]:
output_buffer_4o_mini = io.StringIO()

In [ ]:
for chunk in llm_4omini.stream(prompt_structure_extract_json):
    print(chunk.content, end="", flush=True)  # Affichage en temps réel
    output_buffer_4o_mini.write(chunk.content)  # Stockage dans le buffer



```json
{
  "Article": {
    "article_1": {
      "paragraphe_1": {
        "contenu": "1. La présente directive vise à prévenir l'utilisation du système financier de l'Union aux fins du blanchiment de capitaux et du financement du terrorisme."
      },
      "paragraphe_2": {
        "contenu": "2. Les États membres veillent à ce que le blanchiment de capitaux et le financement du terrorisme soient interdits."
      },
      "paragraphe_3": {
        "contenu": "3. Aux fins de la présente directive, sont considérés comme blanchiment de capitaux les agissements ci-après énumérés, commis intentionnellement:"
      },
      "paragraphe_3_a": {
        "contenu": "a) la conversion ou le transfert de biens, dont celui qui s'y livre sait qu'ils proviennent d'une activité criminelle ou d'une participation à une activité criminelle, dans le but de dissimuler ou de déguiser l'origine illicite de ces biens ou d'aider toute personne impliquée dans une telle activité à échapper aux conséquences j

In [ ]:
# Récup
result_text_extract_4o_mini = output_buffer_4o_mini.getvalue()

In [ ]:
count_tokens(result_text_extract_4o_mini)

16384

In [ ]:
def agentic_chunking(text_data):
    prompt = """I am providing a document below. 
    Please split the document into chunks that maintain semantic coherence and ensure that each chunk represents a complete and meaningful unit of information. 
    Each chunk should stand alone, preserving the context and meaning without splitting key ideas across chunks. 
    Use your understanding of the content’s structure, topics, and flow to identify natural breakpoints in the text. 
    Ensure that no chunk exceeds 1000 characters length, and prioritize keeping related concepts or sections together.

    Do not modify the document, just split to chunks and return them as an array of strings, where each string is one chunk of the document.
    Return the entire book not dont stop in betweek some sentences.

    Document:
    {document}
    """

    prompt_template = PromptTemplate.from_template(prompt)

    chain = prompt_template | llm

    result = chain.invoke({"document": text_data})
    return result

#### 5.4.3 - Data Loader (XL) Langchain

In [ ]:
from langchain_community.document_loaders import UnstructuredExcelLoader

In [ ]:
loader = UnstructuredExcelLoader("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_input/AML_5.xlsx", mode="elements")
docs = loader.load()

print(len(docs))

docs

226


[Document(metadata={'source': '/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_input/AML_5.xlsx', 'file_directory': '/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_input', 'filename': 'AML_5.xlsx', 'last_modified': '2025-03-14T16:23:55', 'page_name': 'Feuil1', 'page_number': 1, 'languages': ['fra'], 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'category': 'Title', 'element_id': 'cacbd36f7f07b497a09099de64991a19'}, page_content='AML5'),
 Document(metadata={'source': '/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_input/AML_5.xlsx', 'file_directory': '/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_input', 'filename': 'AML_5.xlsx', 'last_modified': '2025-03-14T16:23:55', 'page_name': 'Feuil1', 'page_number': 1, 'languages': ['fra'], 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'parent_id': 'cacbd36f7f07b497a09099de64991a19', 'category': 'NarrativeText', 'element_id': '09136bfb847340ab5c817ad1b1e13b3

#### 5.4.4 - Embedding

In [236]:
from langchain.vectorstores import Chroma